In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.metrics import *
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='selu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='selu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation= 'selu' ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [9]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [11]:
run_experiment(X,y)

Epoch 1/10
29/29 [==============================] - 1s 18ms/step - loss: 4.3274 - accuracy: 0.5433 - precision: 0.5188 - recall: 0.3551 - auc: 0.5815
Epoch 2/10
29/29 [==============================] - 1s 18ms/step - loss: 0.7528 - accuracy: 0.6068 - precision: 0.5835 - recall: 0.5631 - auc: 0.6838
Epoch 3/10
29/29 [==============================] - 1s 18ms/step - loss: 0.5316 - accuracy: 0.7503 - precision: 0.7212 - recall: 0.7617 - auc: 0.8648
Epoch 4/10
29/29 [==============================] - 1s 18ms/step - loss: 0.3055 - accuracy: 0.8729 - precision: 0.8545 - recall: 0.8785 - auc: 0.9503
Epoch 5/10
29/29 [==============================] - 1s 18ms/step - loss: 0.2258 - accuracy: 0.8949 - precision: 0.8562 - recall: 0.9322 - auc: 0.9726
Epoch 6/10
29/29 [==============================] - 1s 18ms/step - loss: 0.1545 - accuracy: 0.9288 - precision: 0.9153 - recall: 0.9346 - auc: 0.9863
Epoch 7/10
29/29 [==============================] - 1s 18ms/step - loss: 0.1196 - accuracy: 0.9441 -

In [12]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='selu', input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64, kernel_size=3, activation='selu'))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64, kernel_size=3, activation='selu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation= 'selu' ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [13]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [14]:
run_experiment_2(X,y)

Epoch 1/10
29/29 [==============================] - 1s 24ms/step - loss: 13.6886 - accuracy: 0.5772 - precision_5: 0.5457 - recall_5: 0.5864 - auc_5: 0.5726
Epoch 2/10
29/29 [==============================] - 1s 23ms/step - loss: 0.8576 - accuracy: 0.6791 - precision_5: 0.6433 - recall_5: 0.7079 - auc_5: 0.7440
Epoch 3/10
29/29 [==============================] - 1s 23ms/step - loss: 0.5770 - accuracy: 0.7295 - precision_5: 0.6828 - recall_5: 0.7897 - auc_5: 0.8093
Epoch 4/10
29/29 [==============================] - 1s 23ms/step - loss: 0.4872 - accuracy: 0.7952 - precision_5: 0.7625 - recall_5: 0.8178 - auc_5: 0.8755
Epoch 5/10
29/29 [==============================] - 1s 24ms/step - loss: 0.3520 - accuracy: 0.8499 - precision_5: 0.8392 - recall_5: 0.8411 - auc_5: 0.9253
Epoch 6/10
29/29 [==============================] - 1s 24ms/step - loss: 0.3190 - accuracy: 0.8719 - precision_5: 0.8448 - recall_5: 0.8902 - auc_5: 0.9375
Epoch 7/10
29/29 [==============================] - 1s 24ms/ste

In [15]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='selu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='selu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation= 'selu' ))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation= 'selu' ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [16]:
def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_3(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [17]:
run_experiment_3(X,y)

Epoch 1/10
29/29 [==============================] - 1s 18ms/step - loss: 3.7835 - accuracy: 0.6210 - precision_10: 0.5976 - recall_10: 0.5864 - auc_10: 0.6638
Epoch 2/10
29/29 [==============================] - 1s 17ms/step - loss: 0.7733 - accuracy: 0.7590 - precision_10: 0.7396 - recall_10: 0.7500 - auc_10: 0.8388
Epoch 3/10
29/29 [==============================] - 1s 18ms/step - loss: 0.3799 - accuracy: 0.8653 - precision_10: 0.8474 - recall_10: 0.8692 - auc_10: 0.9407
Epoch 4/10
29/29 [==============================] - 1s 18ms/step - loss: 0.1961 - accuracy: 0.9387 - precision_10: 0.9247 - recall_10: 0.9463 - auc_10: 0.9809
Epoch 5/10
29/29 [==============================] - 1s 18ms/step - loss: 0.1393 - accuracy: 0.9529 - precision_10: 0.9466 - recall_10: 0.9533 - auc_10: 0.9893
Epoch 6/10
29/29 [==============================] - 1s 18ms/step - loss: 0.1291 - accuracy: 0.9715 - precision_10: 0.9741 - recall_10: 0.9650 - auc_10: 0.9885
Epoch 7/10
29/29 [============================